In [ ]:
#
# Hint: If lines are created with support of a Large Language Model or the code is taken from another source, you find following hint at the end of the line:
#       (generated with Microsoft Copilot) or (source: link_to_source)

In [4]:
import json
from dotenv import load_dotenv
from snowflake.snowpark.session import Session
from snowflake.cortex import Complete
import os

In [13]:
load_dotenv(override=True)

# set up connection parameters for Snowflake connection
connection_params = {
    "account": os.environ['SNOWFLAKE_ACCOUNT'],
    "user": os.environ['SNOWFLAKE_USER'],
    "password": os.environ['SNOWFLAKE_USER_PASSWORD'],
    "role": 'ACCOUNTADMIN',
    "warehouse": 'COMPUTE_WH',
    'paramstyle': 'qmark'
}

In [14]:
model = 'llama3.1-8b'
model_summary_params = {
   'temperature': 0, # default: 0 https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex --> Internetrecherche hat keine anderen Empfehlungen ergeben
   # 'top_p': # default: 0 https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex
    'max_tokens': 4000
}

In [15]:
snowflake_session = Session.builder.configs(connection_params).create()
print('Snowflake sessions is build.')
print('---------------------------------------------')

Snowflake sessions is build.
---------------------------------------------


In [16]:
with open('../data/input_readme_data/jianchang512_stt.json', 'r')  as file:
    loaded_data = json.load(file)

source_code_cleaned_comments = loaded_data['source_code_cleaned_comments']
license = loaded_data['license']
requirements = loaded_data['requirements']

input_txt = source_code_cleaned_comments

repo_name = 'stt'
repo_owner = 'jianchang512'
# license = ''
# requirements = ''

In [24]:
def write_summary_prompt(repo_name, input_txt):
    prompt_summary = f'''
        You are acting as a software development expert for the following GitHub repository "{repo_name}".
        Your task is to summarize the given source code string "{input_txt}" in natural language, so a specialist is able to understand
        the purpose of the repository.
        Identify its purpose, key functions, main components and dependencies. Focus on the overall architecture and structure 
        rather than line-by-line details. Do not add any recommendations or improvement suggestions, but concentrate on the summary. 
        Present the summary in a clear and concise language.
        You are not allowed to add any small talk. 
    ''' 
    #prompt_summary = prompt_summary.replace("'", "\\'").replace(";", "\\;").replace('"', '\\"')

    return prompt_summary

In [25]:
prompt_summary = write_summary_prompt(repo_name, input_txt)

https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.Session.sql  
Doku für bind variabels

In [21]:
query = f"""
            SELECT SNOWFLAKE.CORTEX.COMPLETE(
                ?,
                [
                    {{
                        'role': 'user', 
                        'content': ?
                    }}
                ],
                {{
                    'temperature': ?,
                    'max_tokens':  ?
                }} 
            ) AS response
        """

In [26]:
response = snowflake_session.sql(query, params=[model, prompt_summary, model_summary_params['temperature'], model_summary_params['max_tokens']]).collect()

In [ ]:
# query = f"""
#             SELECT SNOWFLAKE.CORTEX.COMPLETE(
#                 '{model}',
#                 [
#                     {{
#                         'role': 'user', 
#                         'content': '{prompt}'
#                     }}
#                 ],
#                 {{
#                     'temperature': {model_params['temperature']},
#                     'max_tokens':  {model_params['max_tokens']}
#                 }} 
#             ) AS response
#         """

In [27]:
# response = snowflake_session.sql(query).collect()
res = json.loads(response[0]['RESPONSE'])

In [28]:
res

{'choices': [{'messages': '**Repository Purpose:**\nThe repository is a web application for automatic speech recognition (ASR) and text-to-speech (TTS) tasks. It utilizes the Whisper model, a state-of-the-art ASR model, and provides a user-friendly interface for uploading audio files, selecting models, and configuring settings.\n\n**Key Functions:**\n\n1. **Audio Upload**: Users can upload audio files in various formats, and the application will convert them to WAV format.\n2. **Model Selection**: Users can choose from a list of pre-trained models for ASR and TTS tasks.\n3. **Configuration**: Users can configure settings such as device type (CPU or CUDA), beam size, and temperature.\n4. **ASR and TTS Processing**: The application will process the uploaded audio file using the selected model and settings, and return the recognized text or synthesized speech.\n5. **Progress Bar**: The application displays a progress bar to indicate the status of the processing task.\n\n**Main Components:

In [29]:
message = res['choices'][0]['messages']

In [32]:
print(message)

**Repository Purpose:**
The repository is a web application for automatic speech recognition (ASR) and text-to-speech (TTS) tasks. It utilizes the Whisper model, a state-of-the-art ASR model, and provides a user-friendly interface for uploading audio files, selecting models, and configuring settings.

**Key Functions:**

1. **Audio Upload**: Users can upload audio files in various formats, and the application will convert them to WAV format.
2. **Model Selection**: Users can choose from a list of pre-trained models for ASR and TTS tasks.
3. **Configuration**: Users can configure settings such as device type (CPU or CUDA), beam size, and temperature.
4. **ASR and TTS Processing**: The application will process the uploaded audio file using the selected model and settings, and return the recognized text or synthesized speech.
5. **Progress Bar**: The application displays a progress bar to indicate the status of the processing task.

**Main Components:**

1. **Flask Web Server**: The appli

In [34]:
snowflake_session.close()
print('---------------------------------------------')
print('Snowflake session is closed.')

---------------------------------------------
Snowflake session is closed.
